# How to read data from DataFrames with pyActigraphy

## Imported packages and input data

The usual suspects:

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import pyActigraphy

/usr/local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In this example, let's generate some input data:

NB: if you already have your data under a pandas.DataFrame format, jump directly to the next section.

In [4]:
N = 1440*7 # 7 days of acquisition at a frequency of 60s.

In [5]:
activity = np.random.normal(10,1,N)
light = np.random.normal(100,10,N)

In [6]:
non_wear = np.empty(N)

In [7]:
# Set up a segment of  spurious inactivity
activity[2060:2160] = 0.0
non_wear[2060:2160] = 1.0

In [8]:
d = {'Activity': activity, 'Light': light, 'Non-wear': non_wear}

In [9]:
index = pd.date_range(start='01-01-2020',freq='60s',periods=N)

In [10]:
data = pd.DataFrame(index=index,data=d)

In [11]:
data

,Activity,Light,Non-wear
2020-01-01 00:00:00,9.803906,97.208266,0.0
2020-01-01 00:01:00,11.322234,106.991005,0.0
2020-01-01 00:02:00,8.204245,99.137269,0.0
2020-01-01 00:03:00,8.867003,99.459600,0.0
2020-01-01 00:04:00,9.187778,90.814416,0.0
...,...,...,...
2020-01-07 23:55:00,7.516059,83.467992,0.0
2020-01-07 23:56:00,10.156999,91.532551,0.0
2020-01-07 23:57:00,10.241579,96.630307,0.0
2020-01-07 23:58:00,10.751310,97.703363,0.0


## Manual creation of a BaseRaw object

In [12]:
from pyActigraphy.io import BaseRaw

In [13]:
help(BaseRaw)

Help on class BaseRaw in module pyActigraphy.io.base:

class BaseRaw(pyActigraphy.sleep.sleep.SleepBoutMixin, pyActigraphy.sleep.scoring_base.ScoringMixin, pyActigraphy.metrics.metrics.MetricsMixin, pyActigraphy.filters.filters.FiltersMixin)
 |  BaseRaw(name, uuid, format, axial_mode, start_time, period, frequency, data, light)
 |  
 |  Base class for raw data.
 |  
 |  Method resolution order:
 |      BaseRaw
 |      pyActigraphy.sleep.sleep.SleepBoutMixin
 |      pyActigraphy.sleep.scoring_base.ScoringMixin
 |      pyActigraphy.metrics.metrics.MetricsMixin
 |      pyActigraphy.filters.filters.FiltersMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, uuid, format, axial_mode, start_time, period, frequency, data, light)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  binarized_data(self, threshold)
 |      Boolean thresholding of Pandas Series
 |  
 |  duration(self)
 |      Duration (in days, hours, etc) of the da

### Set activity and light data (if available)

In [14]:
raw = BaseRaw(
    name="myName", 
    uuid='DeviceId', 
    format='Pandas', 
    axial_mode=None,
    start_time=data.index[0],
    period=(data.index[-1]-data.index[0]),
    frequency=data.index.freq,
    data=data['Activity'],
    light=data['Light']
)

In [15]:
raw.data

2020-01-01 00:00:00     9.803906
2020-01-01 00:01:00    11.322234
2020-01-01 00:02:00     8.204245
2020-01-01 00:03:00     8.867003
2020-01-01 00:04:00     9.187778
                         ...    
2020-01-07 23:55:00     7.516059
2020-01-07 23:56:00    10.156999
2020-01-07 23:57:00    10.241579
2020-01-07 23:58:00    10.751310
2020-01-07 23:59:00     9.172097
Freq: 60S, Name: Activity, Length: 10080, dtype: float64

### Set up a mask

Most devices that have a wear sensor return this information as a binary time series with "1" when the device is most likely not worn and "0" otherwise.
In pyActigraphy, this information can be used to create a mask and thus invalidate the corresponding data points (set to "0" most probably). However, the mask, the value "1" correspond to "no masking". So, depending on your "non-wear" data, be careful to transform them appropriately:

In [16]:
# Here, I assume that 0: the device is worn, 1: device not worn. 
# As mentioned aboce, for the mask, 1: no masking. (NB: it is a convolution: data*mask)
raw.mask = np.abs(data['Non-wear']-1)

## Tests

In [17]:
raw.duration()

<604800 * Seconds>

In [18]:
raw.length()

10080

In [19]:
raw.ADAT(binarize=False)

14254.226390926031

In [20]:
raw.IV(binarize=False)

1.0947050893977417

In [21]:
# If you want to mask the data
raw.mask_inactivity = True

In [22]:
# For a gaussian noise, IV should be close to 2.
raw.IV(binarize=False)

2.0822912245090293

The masking seems to work!

Et voilà!